In [ ]:
#Importing libraries
import pandas as pd
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
import numpy as np
import sdt.changepoint as c
import math

In [ ]:
#import ARIMA libraries
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from statsmodels.graphics import tsaplots
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
import numpy as np

In [ ]:
# create a dataframe from the csv file
df_new_cases = pd.read_csv('../data/WHO-COVID-19-global-data.csv')
#df_vaccination = pd.read_csv('../data/owid-covid-data.csv')
df_vaccination = pd.read_csv('../data/country_vaccinations.csv')

In [ ]:
# add moving average for new cases
#df_vaccination['moving_average_new_vaccinations'] = df_vaccination.groupby('iso_code')['new_vaccinations'].transform(lambda x: x.rolling(window=7).mean()) # OWID
df_vaccination['moving_average_new_vaccinations'] = df_vaccination.groupby('iso_code')['daily_vaccinations'].transform(lambda x: x.rolling(window=7).mean())

# rimuovi tutte le righe che non hanno un valore o hanno un valore a "0.0" per moving_average_new_cases
df_vaccination = df_vaccination.dropna(subset=['moving_average_new_vaccinations'])
df_vaccination = df_vaccination[df_vaccination['moving_average_new_vaccinations'] != 0.0]

In [ ]:
# add moving average for new cases
df_new_cases['moving_average_new_cases'] = df_new_cases.groupby('Country_code')['New_cases'].transform(lambda x: x.rolling(window=7).mean())

# rimuovi tutte le righe che non hanno un valore o hanno un valore a "0.0" per moving_average_new_cases
df_new_cases = df_new_cases.dropna(subset=['moving_average_new_cases'])
df_new_cases = df_new_cases[df_new_cases['moving_average_new_cases'] != 0.0]

In [ ]:
# firter data from 2021-07-16 to 2021-08-15
start_date = date(2021, 1, 1) 
og_start_date = date(2021, 7, 22)
og_end_date = date(2021, 8, 8)
end_date = date(2021, 8, 23)

difference = og_start_date - start_date
event_duration = og_end_date - og_start_date

# filter data from 2021-07-16 to 2021-08-15
#df_new_cases = df_new_cases[(df_new_cases['Date_reported'] >= str(start_date)) & (df_new_cases['Date_reported'] <= str(end_date))]

# filter df_vaccination from 2021-07-16 to 2021-08-15
#df_vaccination = df_vaccination[(df_vaccination['date'] >= str(start_date)) & (df_vaccination['date'] <= str(end_date))]


# filter df_vaccination and df_new_cases from 2020-12-08 to 2021-08-31
start_vaccination = date(2020, 12, 8)
end_study = date(2021, 8, 23)
df_vaccination = df_vaccination[(df_vaccination['date'] >= str(start_vaccination)) & (df_vaccination['date'] <= str(end_study))]
df_vaccination = df_vaccination.reset_index(drop=True)
df_new_cases = df_new_cases[(df_new_cases['Date_reported'] >= str(start_vaccination)) & (df_new_cases['Date_reported'] <= str(end_study))]
df_new_cases = df_new_cases.reset_index(drop=True)

In [ ]:
#filter by iso_cod that not contain "OWID" prefix
df_vaccination = df_vaccination[~df_vaccination['iso_code'].str.contains("OWID")]

# aggragate by iso_code and date and sum total_cases, total_deaths, population, total_vaccinations, new_cases, mind positive_rate, max positive_rate,KEEP LOCATION 
#df_vaccination = df_vaccination.groupby(['iso_code', 'date']).agg({'people_vaccinated':'sum', 'new_vaccinations': 'sum', 'location': 'max', 'moving_average_new_vaccinations': 'sum'})
df_vaccination = df_vaccination.groupby(['iso_code', 'date']).agg({'total_vaccinations':'sum', 'daily_vaccinations': 'sum', 'country': 'max', 'moving_average_new_vaccinations': 'sum'})

In [ ]:
#aggiungi alla riga attuale nella colonna iso_code il valore iso
convert_ISO_3166_2_to_1 = {
    'AF':'AFG',
    'AX':'ALA',
    'AL':'ALB',
    'DZ':'DZA',
    'AS':'ASM',
    'AD':'AND',
    'AO':'AGO',
    'AI':'AIA',
    'AQ':'ATA',
    'AG':'ATG',
    'AR':'ARG',
    'AM':'ARM',
    'AW':'ABW',
    'AU':'AUS',
    'AT':'AUT',
    'AZ':'AZE',
    'BS':'BHS',
    'BH':'BHR',
    'BD':'BGD',
    'BB':'BRB',
    'BY':'BLR',
    'BE':'BEL',
    'BZ':'BLZ',
    'BJ':'BEN',
    'BM':'BMU',
    'BT':'BTN',
    'BO':'BOL',
    'BA':'BIH',
    'BW':'BWA',
    'BV':'BVT',
    'BR':'BRA',
    'IO':'IOT',
    'BN':'BRN',
    'BG':'BGR',
    'BF':'BFA',
    'BI':'BDI',
    'KH':'KHM',
    'CM':'CMR',
    'CA':'CAN',
    'CV':'CPV',
    'KY':'CYM',
    'CF':'CAF',
    'TD':'TCD',
    'CL':'CHL',
    'CN':'CHN',
    'CX':'CXR',
    'CC':'CCK',
    'CO':'COL',
    'KM':'COM',
    'CG':'COG',
    'CD':'COD',
    'CK':'COK',
    'CR':'CRI',
    'CI':'CIV',
    'HR':'HRV',
    'CU':'CUB',
    'CY':'CYP',
    'CZ':'CZE',
    'DK':'DNK',
    'DJ':'DJI',
    'DM':'DMA',
    'DO':'DOM',
    'EC':'ECU',
    'EG':'EGY',
    'SV':'SLV',
    'GQ':'GNQ',
    'ER':'ERI',
    'EE':'EST',
    'ET':'ETH',
    'FK':'FLK',
    'FO':'FRO',
    'FJ':'FJI',
    'FI':'FIN',
    'FR':'FRA',
    'GF':'GUF',
    'PF':'PYF',
    'TF':'ATF',
    'GA':'GAB',
    'GM':'GMB',
    'GE':'GEO',
    'DE':'DEU',
    'GH':'GHA',
    'GI':'GIB',
    'GR':'GRC',
    'GL':'GRL',
    'GD':'GRD',
    'GP':'GLP',
    'GU':'GUM',
    'GT':'GTM',
    'GG':'GGY',
    'GN':'GIN',
    'GW':'GNB',
    'GY':'GUY',
    'HT':'HTI',
    'HM':'HMD',
    'VA':'VAT',
    'HN':'HND',
    'HK':'HKG',
    'HU':'HUN',
    'IS':'ISL',
    'IN':'IND',
    'ID':'IDN',
    'IR':'IRN',
    'IQ':'IRQ',
    'IE':'IRL',
    'IM':'IMN',
    'IL':'ISR',
    'IT':'ITA',
    'JM':'JAM',
    'JP':'JPN',
    'JE':'JEY',
    'JO':'JOR',
    'KZ':'KAZ',
    'KE':'KEN',
    'KI':'KIR',
    'KP':'PRK',
    'KR':'KOR',
    'KW':'KWT',
    'KG':'KGZ',
    'LA':'LAO',
    'LV':'LVA',
    'LB':'LBN',
    'LS':'LSO',
    'LR':'LBR',
    'LY':'LBY',
    'LI':'LIE',
    'LT':'LTU',
    'LU':'LUX',
    'MO':'MAC',
    'MK':'MKD',
    'MG':'MDG',
    'MW':'MWI',
    'MY':'MYS',
    'MV':'MDV',
    'ML':'MLI',
    'MT':'MLT',
    'MH':'MHL',
    'MQ':'MTQ',
    'MR':'MRT',
    'MU':'MUS',
    'YT':'MYT',
    'MX':'MEX',
    'FM':'FSM',
    'MD':'MDA',
    'MC':'MCO',
    'MN':'MNG',
    'ME':'MNE',
    'MS':'MSR',
    'MA':'MAR',
    'MZ':'MOZ',
    'MM':'MMR',
    'NA':'NAM',
    'NR':'NRU',
    'NP':'NPL',
    'NL':'NLD',
    'AN':'ANT',
    'NC':'NCL',
    'NZ':'NZL',
    'NI':'NIC',
    'NE':'NER',
    'NG':'NGA',
    'NU':'NIU',
    'NF':'NFK',
    'MP':'MNP',
    'NO':'NOR',
    'OM':'OMN',
    'PK':'PAK',
    'PW':'PLW',
    'PS':'PSE',
    'PA':'PAN',
    'PG':'PNG',
    'PY':'PRY',
    'PE':'PER',
    'PH':'PHL',
    'PN':'PCN',
    'PL':'POL',
    'PT':'PRT',
    'PR':'PRI',
    'QA':'QAT',
    'RE':'REU',
    'RO':'ROU',
    'RU':'RUS',
    'RW':'RWA',
    'BL':'BLM',
    'SH':'SHN',
    'KN':'KNA',
    'LC':'LCA',
    'MF':'MAF',
    'PM':'SPM',
    'VC':'VCT',
    'WS':'WSM',
    'SM':'SMR',
    'ST':'STP',
    'SA':'SAU',
    'SN':'SEN',
    'RS':'SRB',
    'SC':'SYC',
    'SL':'SLE',
    'SG':'SGP',
    'SK':'SVK',
    'SI':'SVN',
    'SB':'SLB',
    'SO':'SOM',
    'ZA':'ZAF',
    'GS':'SGS',
    'ES':'ESP',
    'LK':'LKA',
    'SD':'SDN',
    'SR':'SUR',
    'SJ':'SJM',
    'SZ':'SWZ',
    'SE':'SWE',
    'CH':'CHE',
    'SY':'SYR',
    'TW':'TWN',
    'TJ':'TJK',
    'TZ':'TZA',
    'TH':'THA',
    'TL':'TLS',
    'TG':'TGO',
    'TK':'TKL',
    'TO':'TON',
    'TT':'TTO',
    'TN':'TUN',
    'TR':'TUR',
    'TM':'TKM',
    'TC':'TCA',
    'TV':'TUV',
    'UG':'UGA',
    'UA':'UKR',
    'AE':'ARE',
    'GB':'GBR',
    'US':'USA',
    'UM':'UMI',
    'UY':'URY',
    'UZ':'UZB',
    'VU':'VUT',
    'VE':'VEN',
    'VN':'VNM',
    'VG':'VGB',
    'VI':'VIR',
    'WF':'WLF',
    'EH':'ESH',
    'YE':'YEM',
    'ZM':'ZMB',
    'ZW':'ZWE'
}

In [ ]:
# set country_code like index
df_new_cases = df_new_cases.set_index('Country_code')
country_list = df_new_cases.index.get_level_values('Country_code').unique()

for code in country_list:
    if code not in convert_ISO_3166_2_to_1:
        #print(code , "NOT FOUND")
        continue
    iso = convert_ISO_3166_2_to_1[code]
    df_new_cases.loc[(df_new_cases.index.get_level_values('Country_code') == code), 'iso_code'] = iso

In [ ]:
# rename columns of date_reported
df_new_cases.rename(columns={'Date_reported':'date'}, inplace=True) 

In [ ]:
# aggragate by iso_code and date and sum total_cases, total_deaths, population, total_vaccinations, new_cases, mind positive_rate, max positive_rate,KEEP LOCATION 
df_new_cases = df_new_cases.groupby(['iso_code', 'date']).agg({'Country': 'sum', 'New_cases': 'sum', 'moving_average_new_cases': 'sum'})

In [ ]:
# save dataframe in csv file
df_new_cases.to_csv('./out/ARIMA/ARIMA_dataframe.csv')
df_vaccination.to_csv('./out/ARIMA/ARIMA_vaccinations.csv')

In [ ]:
'''for iso_code in df_vaccination.index.get_level_values('iso_code').unique():
    plt.plot(df_vaccination.loc[iso_code]['moving_average_new_vaccinations'], label='vaccinations')
    plt.title(df_vaccination.loc[iso_code]['country'].unique()[0])
    plt.ylabel("new vaccinations")
    plt.xlabel("Date")
    plt.xticks(rotation=90)
    plt.show()'''

In [ ]:

# crea una lista di tutti gli iso_code presenti in df_new_cases
iso_code_list = df_new_cases.index.get_level_values('iso_code').unique().tolist()

In [ ]:
def train_model(df_train, df_test):
    min_rmse = 10000000
    best_model = None
    best_d = 0
    best_pred = None
    for diff in range(0, 3):
        model = pm.auto_arima(df_train["moving_average_new_cases"], X=df_train[["moving_average_new_vaccinations"]], start_p=0, start_q=0,
                              # usiamo il test adf per la stazionarietà.
                              test='adf',
                              max_p=5, max_q=5,
                              d=diff,  # ordine della prima differenziazione
                              error_action='ignore', trace=True,
                              suppress_warnings=True,
                              random_state=42,
                              stagionality=True,
                              maxiter=500, m=4, stationary=False)

        pred, ci = model.predict(n_periods=len(
            df_test["moving_average_new_cases"]), X=df_test[["moving_average_new_vaccinations"]], alpha=0.05, return_conf_int=True)

        rmse = math.sqrt(mean_squared_error(
            df_test["moving_average_new_cases"], pred))
        # print('Test RMSE: %.3f' % rmse)
        if rmse < min_rmse:
            min_rmse = rmse
            best_model = model
            best_pred = pred
            best_d = diff
    return best_model, best_pred, best_d, min_rmse

In [ ]:
tabella = pd.DataFrame(columns=['State','Full','Before','During','After'])
trend_negative = []
trend_positive = []
split_date = date(2021, 7, 7)

print(iso_code_list)

for code in iso_code_list:
    try:
        # STEP 1: crea un df con solo data e valore per ogni stato
        df_cases = pd.DataFrame(df_new_cases.loc[code]['moving_average_new_cases'].values, index = df_new_cases.loc[code]['moving_average_new_cases'].index, columns = ['moving_average_new_cases'])
        df_vaccin = pd.DataFrame(df_vaccination.loc[code]['moving_average_new_vaccinations'].values, index = df_vaccination.loc[code]['moving_average_new_vaccinations'].index, columns = ['moving_average_new_vaccinations'])
        
        # STEP 2: mergia i due df in uno solo
        merged_df = df_cases.merge(df_vaccin, on=['date'])
        merged_df.to_csv(f"./out/ARIMA/tmp/merged_df_{code}.csv")
        
        #find split date for df_vaccin that indicate first value not null of 0 for moving_average_new_vaccinations
        first_date = merged_df.loc[merged_df['moving_average_new_vaccinations'] > 0].index[0]
        last_date = str(split_date.strftime("%Y-%m-%d"))
        
        merged_df_train = merged_df.loc[(merged_df.index >= first_date) & (merged_df.index <= last_date)].copy()
        merged_df_test = merged_df.loc[merged_df.index > last_date].copy()
        
        print(merged_df_train)
        print(merged_df_test)
        
        
        best_model, best_pred, best_d, min_rmse = train_model(merged_df_train, merged_df_test)
        
        # STEP 4: crea un modello auto_ARIMA per predirre i new_cases considerando i vaccini fino 2 settimane prima dell'inizio delle olimpiadi
        model = pm.auto_arima(merged_df_train['moving_average_new_cases'], X=merged_df_train[["moving_average_new_vaccinations"]], start_p=0, start_q=0,
                                    test='adf',       # use adftest to find optimal 'd'
                                    max_p=3, max_q=3, # maximum p and q
                                    d=best_d,          # let model determine 'd'
                                    m=4,              # frequency of series
                                    error_action='ignore',
                                    stagionality=True,
                                    suppress_warnings=True,
                                    random_state=42,
                                    maxiter=500, stationary=False,
                                    trace=True,)
        
        length = len(merged_df_test)
        pred, ci = model.predict(n_periods=length, X=merged_df_test[["moving_average_new_vaccinations"]], alpha=0.05, return_conf_int=True)
        print("------------------")
        print(pred)
        print("------------------")
        
        #STEP 5: dividi il test set in 3 parti: 14 giorni prima, durante le olimpiadi e 14 giorni dopo    
        before_olympic = merged_df_test.loc[merged_df_test.index <= str(split_date + timedelta(days=14))].copy()
        during_olympic = merged_df_test.loc[(merged_df_test.index > str(split_date + timedelta(days=14))) & (merged_df_test.index <= str(split_date + timedelta(days=21)))].copy()
        after_olympic = merged_df_test.loc[merged_df_test.index > str(split_date + timedelta(days=21))].copy()
        
        #STEP 6: predici i new_cases per le 3 parti del test set
        full_olympic_predicted = model.predict(n_periods=len(merged_df_test), X=merged_df_test[["moving_average_new_vaccinations"]])
        before_olympic_predicted = full_olympic_predicted[:14]
        during_olympic_predicted = full_olympic_predicted[14:21]
        after_olympic_predicted = full_olympic_predicted[21:]
        
        #STEP 7: calcola la media dei new_cases predetti per le 3 parti del test set
        full_olympic_mean_predicted = full_olympic_predicted.mean()
        before_olympic_mean_predicted = full_olympic_predicted[:14].mean()
        during_olympic_mean_predicted = full_olympic_predicted[14:21].mean()
        after_olympic_mean_predicted = full_olympic_predicted[21:].mean()
        
        #STEP 8: calcola la differenza tra la media dei new_cases predetti e la media dei new_cases reali per le 3 parti del test set
        full_olympic_difference = full_olympic_mean_predicted - merged_df_test['moving_average_new_cases'].mean()
        before_olympic_difference = before_olympic_mean_predicted - before_olympic['moving_average_new_cases'].mean()
        during_olympic_difference = during_olympic_mean_predicted - during_olympic['moving_average_new_cases'].mean()
        after_olympic_difference = after_olympic_mean_predicted - after_olympic['moving_average_new_cases'].mean()
        
        #STEP 9: calcola la percentuale di differenza tra la media dei new_cases predetti e la media dei new_cases reali per le 3 parti del test set
        full_olympic_percentage_difference = (full_olympic_difference/merged_df_test['moving_average_new_cases'].mean())*100
        before_olympic_percentage_difference = (before_olympic_difference/before_olympic['moving_average_new_cases'].mean())*100
        during_olympic_percentage_difference = (during_olympic_difference/during_olympic['moving_average_new_cases'].mean())*100
        after_olympic_percentage_difference = (after_olympic_difference/after_olympic['moving_average_new_cases'].mean())*100
        
        
        #STEP 10: salva i risultati in un df
        row = pd.DataFrame.from_dict({'State': code, 'Full': full_olympic_percentage_difference, 'Before': before_olympic_percentage_difference, 'During': during_olympic_percentage_difference, 'After': after_olympic_percentage_difference}, orient='index').T
        tabella = pd.concat([tabella, row], ignore_index=True)
        
        #STEP 11: plot the results
        plt.figure(figsize=(15,5))
        plt.plot(merged_df_test.index, merged_df_test['moving_average_new_cases'], label = 'Real new cases')
        plt.plot(merged_df_test.index, full_olympic_predicted, label = 'Predicted new cases')
        plt.title(code + " - " + df_new_cases.loc[code]['Country'].unique()[0])
        plt.legend()
        
        ymax = plt.ylim()[1]
        plt.ylim(ymax=ymax*1.20)
        
        plt.text('2021-07-08', ymax, "14 days before\nfirst game of\nolympic games", rotation=0, color="white", backgroundcolor="blue")
        plt.text('2021-07-22', ymax, "Olympic \nGames", rotation=0, color="black", backgroundcolor="yellow")
        plt.text("2021-07-29", ymax, "7 days \nafter first metch", rotation=0, backgroundcolor="orange")
        plt.text('2021-08-15', ymax, "7 days \nafter last match", rotation=0, backgroundcolor="#FF7D4D")
        
        plt.axvline(x = '2021-07-08', color = 'blue', label = '14 days before first game of olympic games')
        plt.axvspan('2021-07-22', '2021-08-08', facecolor='#ffe206', alpha=0.5)
        plt.axvline(x = '2021-07-29', color = 'orange', label = '7 days after first game of olympic games')
        plt.axvline(x = '2021-08-15', color = '#FF7D4D', label = '7 days after last game of olympic games')
        
        plt.xlabel("Days since the 28th of July 2021")
        plt.xticks(rotation=90)
        plt.ylabel("Current MA - Previous MA")
        
        #save the plot as a png file
        plt.savefig("../pictures/"+ code + "_intervals" +".png")
        plt.savefig("./out/ARIMA/pictures/"+ code + "_intervals" +".png")
        plt.show()
    except:
        print("Error in " + code )

In [ ]:
#tabella is the table for changepoints and tabella2 is the table for the intervals
tabella.set_index(tabella['State'])
tabella.to_csv("./ARIMA_tabella.csv", index=False)